In [13]:
# modules we'll use
import pandas as pd
import numpy as np

# helpful modules
import rapidfuzz
import charset_normalizer

# read in all our data
professors = pd.read_csv("pakistan_intellectual_capital.csv")
professors.head()

,Unnamed: 0,S#,Teacher Name,University Currently Teaching,Department,Province University Located,Designation,Terminal Degree,Graduated from,Country,Year,Area of Specialization/Research Interests,Other Information
0,2,3,Dr. Abdul Basit,University of Balochistan,Computer Science & IT,Balochistan,Assistant Professor,PhD,Asian Institute of Technology,Thailand,NaN,Software Engineering & DBMS,NaN
1,4,5,Dr. Waheed Noor,University of Balochistan,Computer Science & IT,Balochistan,Assistant Professor,PhD,Asian Institute of Technology,Thailand,NaN,DBMS,NaN
2,5,6,Dr. Junaid Baber,University of Balochistan,Computer Science & IT,Balochistan,Assistant Professor,PhD,Asian Institute of Technology,Thailand,NaN,"Information processing, Multimedia mining",NaN
3,6,7,Dr. Maheen Bakhtyar,University of Balochistan,Computer Science & IT,Balochistan,Assistant Professor,PhD,Asian Institute of Technology,Thailand,NaN,"NLP, Information Retrieval, Question Answering...",NaN
4,24,25,Samina Azim,Sardar Bahadur Khan Women's University,Computer Science,Balochistan,Lecturer,BS,Balochistan University of Information Technolo...,Pakistan,2005.0,VLSI Electronics DLD Database,NaN


In [14]:
countries = professors.Country.unique()

countries.sort()
countries

array([' Germany', ' New Zealand', ' Sweden', ' USA', 'Australia',
       'Austria', 'Canada', 'China', 'Finland', 'France', 'Greece',
       'HongKong', 'Ireland', 'Italy', 'Japan', 'Macau', 'Malaysia',
       'Mauritius', 'Netherland', 'New Zealand', 'Norway', 'Pakistan',
       'Portugal', 'Russian Federation', 'Saudi Arabia', 'Scotland',
       'Singapore', 'South Korea', 'SouthKorea', 'Spain', 'Sweden',
       'Thailand', 'Turkey', 'UK', 'USA', 'USofA', 'Urbana', 'germany'],
      dtype=object)

As we can see here, there are some rare instances where a country is repeated with an alternative spelling.

In order to improve the consistency in the data, a few things we can do is make everything lowercase and remove any whitespaces at the beginning and end of the cells as these inconsistencies are very common in text data.

In [15]:
professors.Country = professors.Country.str.lower()
professors.Country = professors.Country.str.strip()
countries = professors.Country.unique()
countries

array(['thailand', 'pakistan', 'germany', 'austria', 'australia', 'uk',
       'china', 'france', 'usofa', 'southkorea', 'malaysia', 'sweden',
       'italy', 'canada', 'norway', 'ireland', 'new zealand', 'urbana',
       'portugal', 'russian federation', 'usa', 'finland', 'netherland',
       'greece', 'turkey', 'south korea', 'macau', 'singapore', 'spain',
       'japan', 'hongkong', 'saudi arabia', 'mauritius', 'scotland'],
      dtype=object)

Fuzzy matching is the process of finding the similarity between text strings based on the amount of characters one would need to change to transform one of the strings to another.

In [16]:
# get the top 10 closest matches to "south korea"
matches = rapidfuzz.process.extract("south korea", countries, limit=10, scorer=rapidfuzz.fuzz.token_sort_ratio)

# take a look at them
matches

[('south korea', 100.0, 25),
 ('southkorea', 47.61904761904761, 9),
 ('saudi arabia', 43.47826086956522, 31),
 ('norway', 35.29411764705882, 14),
 ('austria', 33.333333333333336, 3),
 ('ireland', 33.333333333333336, 15),
 ('pakistan', 31.57894736842105, 1),
 ('portugal', 31.57894736842105, 18),
 ('scotland', 31.57894736842105, 33),
 ('australia', 30.000000000000004, 4)]

In [17]:
def replace_matches_in_column(df, column, string_to_match, min_ratio=47):
    strings = df[column].unique()
    matches = rapidfuzz.process.extract(string_to_match, strings, limit=10, scorer=rapidfuzz.fuzz.token_sort_ratio)
    close_matches = [matches[0] for matches in matches if matches[1] >= min_ratio]
    rows_with_matches = df[column].isin(close_matches)
    df.loc[rows_with_matches, column] = string_to_match

In [18]:
replace_matches_in_column(df=professors, column='Country', string_to_match="south korea")

In [19]:
# get all the unique values in the 'Country' column
countries = professors['Country'].unique()

# sort them alphabetically and then take a closer look
countries.sort()
countries

array(['australia', 'austria', 'canada', 'china', 'finland', 'france',
       'germany', 'greece', 'hongkong', 'ireland', 'italy', 'japan',
       'macau', 'malaysia', 'mauritius', 'netherland', 'new zealand',
       'norway', 'pakistan', 'portugal', 'russian federation',
       'saudi arabia', 'scotland', 'singapore', 'south korea', 'spain',
       'sweden', 'thailand', 'turkey', 'uk', 'urbana', 'usa', 'usofa'],
      dtype=object)

In [20]:
replace_matches_in_column(df=professors, column='Country', string_to_match="usa")
countries = professors['Country'].unique()

# sort them alphabetically and then take a closer look
countries.sort()
countries

array(['canada', 'china', 'finland', 'france', 'germany', 'greece',
       'hongkong', 'ireland', 'italy', 'japan', 'macau', 'malaysia',
       'mauritius', 'netherland', 'new zealand', 'norway', 'pakistan',
       'portugal', 'russian federation', 'saudi arabia', 'scotland',
       'singapore', 'south korea', 'sweden', 'thailand', 'turkey', 'uk',
       'urbana', 'usa'], dtype=object)